In [2]:
pip install scikit-optimize


Note: you may need to restart the kernel to use updated packages.


In [3]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA as randomizedPCA
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt 
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import warnings
from scipy import stats
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.linear_model import  Ridge, ElasticNet, Lasso, LinearRegression, LogisticRegression
from sklearn.model_selection import LeaveOneOut, train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from scipy.stats import norm 
from sklearn.datasets import load_iris
from sklearn.pipeline import Pipeline
import statsmodels.api as sm
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer






In [4]:
data=pd.read_csv("brain_tumor_dataset.csv")
data.head()

,Patient_ID,Age,Gender,Tumor_Type,Tumor_Size,Location,Histology,Stage,Symptom_1,Symptom_2,Symptom_3,Radiation_Treatment,Surgery_Performed,Chemotherapy,Survival_Rate,Tumor_Growth_Rate,Family_History,MRI_Result,Follow_Up_Required
0,1,73,Male,Malignant,5.375612,Temporal,Astrocytoma,III,Vision Issues,Seizures,Seizures,No,No,No,51.312579,0.111876,No,Positive,Yes
1,2,26,Male,Benign,4.847098,Parietal,Glioblastoma,II,Headache,Headache,Nausea,Yes,Yes,Yes,46.373273,2.165736,Yes,Positive,Yes
2,3,31,Male,Benign,5.588391,Parietal,Meningioma,I,Vision Issues,Headache,Seizures,No,No,No,47.072221,1.884228,No,Negative,No
3,4,29,Male,Malignant,1.436600,Temporal,Medulloblastoma,IV,Vision Issues,Seizures,Headache,Yes,No,Yes,51.853634,1.283342,Yes,Negative,No
4,5,54,Female,Benign,2.417506,Parietal,Glioblastoma,I,Headache,Headache,Seizures,No,No,Yes,54.708987,2.069477,No,Positive,Yes


In [5]:
data.describe()

,Patient_ID,Age,Tumor_Size,Survival_Rate,Tumor_Growth_Rate
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,10000.500000,49.433700,5.236552,70.131699,1.547701
std,5773.647028,17.389296,2.754359,17.270834,0.835995
min,1.000000,20.000000,0.500410,40.002386,0.100017
25%,5000.750000,34.000000,2.836686,55.224439,0.830433
50%,10000.500000,49.000000,5.200675,70.305066,1.541821
75%,15000.250000,65.000000,7.624299,85.043395,2.262484
max,20000.000000,79.000000,9.999420,99.998125,2.999835


In [6]:
num_features = data.select_dtypes(include=['number']).columns.tolist()
cat_features = data.select_dtypes(include=['object', 'category']).columns.tolist()
target = ['Tumor_Type'] if 'Tumor_Type' in data.columns else []

Valores numericos:

In [7]:
num_features

['Patient_ID', 'Age', 'Tumor_Size', 'Survival_Rate', 'Tumor_Growth_Rate']

Valores de categorías o texto que representan clases o etiquetas:

In [8]:
cat_features

['Gender',
 'Tumor_Type',
 'Location',
 'Histology',
 'Stage',
 'Symptom_1',
 'Symptom_2',
 'Symptom_3',
 'Radiation_Treatment',
 'Surgery_Performed',
 'Chemotherapy',
 'Family_History',
 'MRI_Result',
 'Follow_Up_Required']

Forma estándar de separar las features del target en un dataset para machine learning:

In [10]:
X=data.drop(columns=target)
y=data[target]

Preprocesar tus columnas numéricas antes de entrenar un modelo:
- Transformar cada columna numérica para que tenga media 0 y desviación estándar 1
- Permite aplicar transformaciones diferentes a distintas columnas de tu dataframe.

In [11]:
numerical_transformer=StandardScaler()
preprocessor=ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_features)]
)

Define SVM → crea pipeline con preprocesamiento → entrena y evalúa con CV → calcula promedio ROC AUC:

(mean_score es el rendimiento esperado del modelo en datos no vistos)

In [13]:
svc= SVC(kernel='linear', C=1, probability=True)
cv=KFold(n_splits=10, shuffle=True, random_state=42)

pipeline=Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('svc', svc)
])
pipeline.fit(X, y)
scores = cross_val_score(pipeline, X, y, cv=10, scoring='roc_auc_ovr')
mean_score=np.mean(scores)

mean_score

c:\Users\luis ramon\anaconda3\ANACONDA\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Optimización bayesiana para buscar el mejor valor del parámetro C de tu SVM usando validación cruzada y optimización bayesiana, en lugar de probar valores manualmente.

opt.best_params_ → te da el mejor valor de C encontrado.

opt.best_score_ → ROC AUC promedio más alto logrado durante la optimización.

In [ ]:
search_spaces = {
   
    'svc__C': Real(1e-4, 1e+4, prior='log-uniform')
}
cv_strategy = KFold(n_splits=10, shuffle=True, random_state=42)
opt = BayesSearchCV(
    estimator=pipeline,
    search_spaces=search_spaces,
    n_iter=50,
    cv=cv_strategy,
    scoring='roc_auc_ovr',
    random_state=42,
    n_jobs=-1,
    verbose=1
)

opt.fit(X, y)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
